In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
# Mount your Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Read the dataset from the specified path
df = pd.read_csv('/content/preprocess_text_Nltk.csv', sep=',', encoding='utf-8', quotechar='"')


In [4]:
# Display the first few rows of the DataFrame
df.head()

text  type  \
0  WASHINGTON (Reuters) - The head of a conservat...  True   
1  WASHINGTON (Reuters) - Transgender people will...  True   
2  WASHINGTON (Reuters) - The special counsel inv...  True   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  True   
4  SEATTLE/WASHINGTON (Reuters) - President Donal...  True   

                                      processed_Text  
0  washington reuters head conservative republica...  
1  washington reuters transgender people allowed ...  
2  washington reuters special counsel investigati...  
3  washington reuters trump campaign adviser geor...  
4  seattlewashington reuters president donald tru...

show only code for  the  method to generate TF-IDF_Vector representations for each text

**1. TF-IDF with scikit-learn's TfidfVectorizer:**

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Function to generate TF-IDF vectors
def generate_tfidf_vectors(texts):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_vectors = tfidf_vectorizer.fit_transform(texts.fillna(''))  # Fill missing values with empty strings
    return tfidf_vectors.toarray()

# Assuming 'processed_Text' column contains the preprocessed text
# Apply the function to generate TF-IDF vectors for each text
df['TF-IDF_Vector'] = generate_tfidf_vectors(df['processed_Text'])

# Display the DataFrame with TF-IDF vector representations
df.head()
#My session crashed after using all available RAM.

NameError: name 'df' is not defined

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import Word2Vec, Doc2Vec, LdaModel
import spacy
from transformers import BertTokenizer, BertModel
import numpy as np

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Define a function to tokenize text using spaCy
def tokenize(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_punct and not token.is_space and not token.is_digit]
    return tokens

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to generate BERT embeddings
def generate_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embeddings

# Function to generate TF-IDF vectors
def generate_tfidf_vectors(texts):
    tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenize)
    tfidf_vectors = tfidf_vectorizer.fit_transform(texts)
    return tfidf_vectors.toarray()

# Function to generate Word2Vec embeddings
def generate_word2vec_embeddings(texts):
    word2vec_model = Word2Vec(sentences=texts, vector_size=100, window=5, min_count=1, workers=4)
    word_vectors = [word2vec_model.wv[word] for word in word2vec_model.wv.key_to_index]
    return np.array(word_vectors)

# Function to generate Doc2Vec embeddings
def generate_doc2vec_embeddings(texts):
    tagged_data = [TaggedDocument(words=tokenize(text), tags=[str(i)]) for i, text in enumerate(texts)]
    doc2vec_model = Doc2Vec(tagged_data, vector_size=100, window=2, min_count=1, workers=4)
    doc_vectors = [doc2vec_model.docvecs[str(i)] for i in range(len(texts))]
    return np.array(doc_vectors)

# Function to generate BERT embeddings
def generate_bert_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embeddings

# Function to generate LDA topic models
def generate_lda_topics(texts):
    tokenized_texts = [tokenize(text) for text in texts]
    dictionary = Dictionary(tokenized_texts)
    corpus = [dictionary.doc2bow(text) for text in tokenized_texts]
    lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)
    topic_vectors = lda_model[corpus]
    return topic_vectors

# Function to generate Bag-of-Words (BoW) vectors using scikit-learn's CountVectorizer
def generate_bow_vectors(texts):
    count_vectorizer = CountVectorizer(tokenizer=tokenize)
    bow_vectors = count_vectorizer.fit_transform(texts)
    return bow_vectors.toarray()

# Function to generate Bag-of-Words (BoW) vectors using Gensim
def generate_gensim_bow_vectors(texts):
    tokenized_texts = [tokenize(text) for text in texts]
    dictionary = Dictionary(tokenized_texts)
    bow_vectors = [dictionary.doc2bow(text) for text in tokenized_texts]
    return bow_vectors

# Apply each method to generate vector representations for each text
df['TF-IDF_Vector'] = generate_tfidf_vectors(df['processed_Text'])
df['Word2Vec_Embedding'] = generate_word2vec_embeddings(df['processed_Text'])
df['Doc2Vec_Embedding'] = generate_doc2vec_embeddings(df['processed_Text'])
df['BERT_Embedding'] = generate_bert_embeddings(df['processed_Text'])
df['LDA_Topics'] = generate_lda_topics(df['processed_Text'])
df['CountVectorizer_BoW'] = generate_bow_vectors(df['processed_Text'])
df['Gensim_BoW'] = generate_gensim_bow_vectors(df['processed_Text'])

# Display the DataFrame with vector representations
df.head()